In [1]:
''' LIBRARIES '''
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re
import time
from IPython.display import clear_output
import multiprocessing
import threading
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

from fuzzywuzzy import process, fuzz
from sympy import symbols, Eq, solve
import pandas as pd
import numpy as np

In [2]:
''' NAVIGATOR BODOG ''' 

url_bod = 'https://www.bodog.com/es/sports/live'

options = Options()

bod =  webdriver.Chrome(r'C:\Users\KNL\Desktop\Ale\Upwork\Scraping_bets\chromedriver.exe', options=options)
bod.get(url_bod)
bod.maximize_window()


In [14]:
path

'/html/body/bx-site/ng-component/div/sp-sports-ui/div/main/div/section/main/sp-path-event/div/div[2]/sp-happening-now/div/div/div/div[1]/sp-coupon/sp-multi-markets/section/article'

In [15]:
path2

'/html/body/bx-site/ng-component/div/sp-sports-ui/div/main/div/section/main/sp-path-event/div/div[2]/sp-happening-now/div/div/div/div[4]/sp-coupon[2]/sp-multi-markets/section/article'

In [39]:
def bodog_live():
    df_bodog = pd.DataFrame(columns=['teams', 'home', 'tie', 'away', 'over', 'goals', 'under', 'site'])
    for j in range(10):
        for k in range(5):
            try:
                elemento = bod.find_element(By.XPATH, f'/html/body/bx-site/ng-component/div/sp-sports-ui/div/main/div/section/main/sp-path-event/div/div[2]/sp-happening-now/div/div/div/div[{j}]/sp-coupon[{k}]/sp-multi-markets/section/article').text.split('\n')
                if len(elemento)>10:
                    teams = elemento[2]+' '+elemento[3]+' '+elemento[4]+' '+elemento[5]+' '+elemento[6]+' '+elemento[7]
                    teams = ''.join([i for i in teams if not i.isdigit()]).replace('+  Apuestas', '')
                    numbers = elemento[-5]+' '+elemento[-4]+' '+elemento[-3]+' '+elemento[-2]+' '+elemento[-1]
                    numbers = numbers.split(' ')

                    home=numbers[0]
                    tie = numbers[1]
                    away = numbers[2]

                    goals = numbers[3]
                    over = numbers[4].replace("(", "").replace(")","")
                    under = numbers[6].replace("(", "").replace(")","")


                    df_bodog.loc[len(df_bodog.index)]=[teams, home, tie, away, over, goals, under, 'bodog']
                    #print(teams,'|', numbers,  f'|| {len(elemento)}', 'j',j,'k', k)

            except:
                pass
    return df_bodog

  Hapoel Kfar Shelem  Beitar Tel Aviv Bat Yam  | ['1.105', '6.25', '26.00', '2.5', '(2.15)', '2.5', '(1.667)'] || 13 j 1 k 1
 Club Almagro (R)  CA Atlanta (R)   | ['2.34', '2.42', '3.70', '3.5', '(2.20)', '3.5', '(1.625)'] || 13 j 2 k 1
  AD Guarabira PB Serra Branca EC PB   | ['29.00', '10.50', '1.029', '5.5', '(1.714)', '5.5', '(2.08)'] || 13 j 3 k 1
  Barracas Central (R)  Independiente (R)  | ['3.55', '2.72', '2.18', '1.5', '(1.676)', '1.5', '(2.13)'] || 15 j 4 k 1
  Estudiantes de LP (R) Huracan (R)   | ['2.64', '2.66', '2.78', '3.5', '(1.893)', '3.5', '(1.847)'] || 14 j 4 k 2
  Patronato (R)  Lanus (R)  | ['3.95', '2.70', '2.08', '1.5', '(1.952)', '1.5', '(1.80)'] || 14 j 4 k 3
  Tensung FC   Ugyen Academy | ['67.00', '51.00', '1.001', '3.5', '(2.20)', '3.5', '(1.625)'] || 15 j 6 k 1
  Persepolis  Mes Rafsanjan  | ['2.20', '2.50', '4.50', '1.5', '(1.87)', '1.5', '(1.87)'] || 13 j 7 k 1
  Coastal Union   Geita Gold | ['3.15', '1.935', '3.60', '0.5', '(1.476)', '0.5', '(2.52)'] || 

,teams,home,tie,away,over,goals,under,site
0,Hapoel Kfar Shelem Beitar Tel Aviv Bat Yam,1.105,6.25,26.00,2.15,2.5,1.667,bodog
1,Club Almagro (R) CA Atlanta (R),2.34,2.42,3.70,2.20,3.5,1.625,bodog
2,AD Guarabira PB Serra Branca EC PB,29.00,10.50,1.029,1.714,5.5,2.08,bodog
3,Barracas Central (R) Independiente (R),3.55,2.72,2.18,1.676,1.5,2.13,bodog
4,Estudiantes de LP (R) Huracan (R),2.64,2.66,2.78,1.893,3.5,1.847,bodog
5,Patronato (R) Lanus (R),3.95,2.70,2.08,1.952,1.5,1.80,bodog
6,Tensung FC Ugyen Academy,67.00,51.00,1.001,2.20,3.5,1.625,bodog
7,Persepolis Mes Rafsanjan,2.20,2.50,4.50,1.87,1.5,1.87,bodog
8,Coastal Union Geita Gold,3.15,1.935,3.60,1.476,0.5,2.52,bodog
9,El Alominiom Al Gounah,3.60,2.40,2.50,2.40,1.5,1.541,bodog


In [32]:
numbers.split(' ')

['1.208', '5.10', '13.00', '2.5', '(1.676)', '2.5', '(2.13)']

In [38]:
j=6
k=1
path = f'/html/body/bx-site/ng-component/div/sp-sports-ui/div/main/div/section/main/sp-path-event/div/div[2]/sp-happening-now/div/div/div/div[{j}]/sp-coupon[{k}]/sp-multi-markets/section/article'
elements = bod.find_element(By.XPATH, path).text.split('\n')
for i, c in enumerate(elements):
    print(i, c)

len(elements)

0 2M
1 77:18
2 0
3 3
4 Tensung FC
5 3
6 1
7 Ugyen Academy
8 6
9 + 9 Apuestas
10 67.00
11 51.00
12 1.001
13 3.5 (2.13)
14 3.5 (1.676)


15

In [11]:
j=2
k=1
path = f'/html/body/bx-site/ng-component/div/sp-sports-ui/div/main/div/section/main/sp-path-event/div/div[2]/sp-happening-now/div/div/div/div[{j}]/sp-coupon[{k}]/sp-multi-markets/section/article'
elements = bod.find_element(By.XPATH, path).text.split('\n')
for i, c in enumerate(elements):
    print(i, c)

len(elements)

0 1M
1 23:43
2 0
3 0
4 Club Almagro (R)
5 3
6 CA Atlanta (R)
7 1
8 + 38 Apuestas
9 2.48
10 2.74
11 2.95
12 1.5 (1.69)
13 1.5 (2.10)


14